# Exercise (part 1) - implementation of FBA problem
## 3.
## a. Evaluate if single gene deletions enhance the production of the compound. <br> Rank the mutants obtained according to the compound production capacity and growth performance. 
### 1º Simulação FBA para saber a taxa de crescimento antes das deleções de genes, ou seja, taxa de crescimento do wild-type <br>
**Model objective - uma vez que temos que considerar que as células evoluíram para cresciento máximo, queremos maximizar a biomassa e, por isso, temos que mudar de 1 para -1; e tbm queremos maximizar a produção do nosso composto (lactato).**

In [ ]:
import cobra

import pandas
from time import time

from cobra.util.solver import linear_reaction_coefficients

import cobra.test
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, double_gene_deletion, double_reaction_deletion)

cobramo = cobra.io.read_sbml_model('iML1515.xml')

cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
cobramo.objective

cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

solution = cobramo.optimize()
print('FBA atraves do cobra:', solution)
print('taxa de crescimento antes da maximização da produção do composto:', solution.objective_value)
print('fluxo da reação da biomassa:', solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

cobramo.objective = {cobramo.reactions.EX_lac__L_e: -1.0, cobramo.reactions.EX_lac__D_e: -1.0}
cobramo.objective

solutionopt = cobramo.optimize()
print('FBA atraves do cobra:', solutionopt)
print('taxa de crescimento depois da maximização da produção do composto:', solutionopt.objective_value)
print('fluxo da reação da biomassa:', solutionopt.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

### 2º (wild type) Reações que (SÓ) produzem  lactato e fluxos delas:

In [ ]:
print('R_EX_lac__L_e', solutionopt.fluxes['EX_lac__L_e'])
print('R_EX_lac__D_e', solutionopt.fluxes['EX_lac__D_e'])

### 3º All single gene deletions

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 2000)

deletion_results = single_gene_deletion(cobramo)
print(deletion_results)

### 4º genes essenciais (não podem ser eliminados):

In [ ]:
from reframed.io.sbml import load_cbmodel
model = load_cbmodel('iML1515.xml', flavor = 'fbc2')

from mewpy.simulation import get_simulator
simul = get_simulator(model)

envcond = {'R_EX_o2_e':0, 'R_EX_glc__D_e':-15}

simul = get_simulator(model, envcond = envcond)

simul.objective = {'R_BIOMASS_Ec_iML1515_core_75p37M': -1.0, 'R_EX_lac__L_e': -1.0, 'R_EX_lac__D_e': -1.0}

result = simul.simulate(method='FBA')
print("FBA atraves do reframed:")
print(result)
print(result.fluxes['R_BIOMASS_Ec_iML1515_core_75p37M'])
print(result.fluxes['R_EX_lac__L_e'])
print(result.fluxes['R_EX_lac__D_e'])

essent_genes = simul.essential_genes
print(essent_genes)

### 5º Iterar sobre cada gene que está no deletion_results, sempre que aparecer um gene essencial continuar o loop e depois apagar gene a gene do modelo. Após tirar um gene, fazer a simulação FBA e calcular os fluxos das reações que produzem lactato. No fim, voltar a colocar o modelo com os genes todos.

In [ ]:
import pandas
from time import time

import cobra.test

import cobra

from mewpy.simulation import get_simulator

for id in deletion_results.ids:
    for gene in id:
        genelist = [str(gene)]
        if gene not in essent_genes:
            print(genelist)
            cobra.manipulation.remove_genes(cobramo, genelist)
            solutionko = cobramo.optimize()
            solutionko.objective_value
            print('produção da lactose', solutionko.objective_value)
            
            cobramo = cobra.io.read_sbml_model('iML1515.xml')
            
            cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
            cobramo.objective

            cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
            cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

            cobra.manipulation.remove_genes(cobramo, genelist)
            solutionkobiomass = cobramo.optimize()
            print('growth rate', solutionkobiomass.objective_value)
            
            cobramo = cobra.io.read_sbml_model('iML1515.xml')

            cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
            cobramo.objective

            cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
            cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

            solution = cobramo.optimize()

            cobramo.objective = {cobramo.reactions.EX_lac__L_e: -1.0, cobramo.reactions.EX_lac__D_e: -1.0}
            cobramo.objective            
